# Эффект Джоуля-Томсона

In [38]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

matplotlib.use('pgf')
matplotlib.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

def read_files(files):
    data = []
    for f in files:
        data += [pd.read_csv(f, sep=',', skipinitialspace=True)]
    return data

T = [17, 30, 40, 50]
colors = ['red', 'green', 'blue', 'magenta']
[data] = read_files(['data/T.csv'])

def plot_linear(x, y, label, color):
    z = np.linspace(min(x), max(x), 1000)
    a, b = np.polyfit(x, y, deg=1)
    plt.plot(z, a * z + b, label=label, color=color)
    return (a, b)
    
plt.figure(figsize=(7, 4))

mu = []
muerr = []
P_ERR = 0.1
# Аппроксимация
for i in range(1, 5):
    y = data[f'T{i}']
    yerr = data[f'dT{i}']
    (a, b) = plot_linear(data['P'], y, 
                         label=f'$T = {T[i - 1]}$ \\textdegree C',
                         color=colors[i - 1])
    aerr = np.sqrt(1 / (data['P'].__len__() - 1) * (sum([(t - b)**2 for t in y]) 
                                                    / sum([p**2 for p in data['P']]) - a**2))
    syserr = a * np.sqrt((P_ERR / (max(data['P']) - min(data['P'])))**2
                         + (np.average(yerr) / (max(y) - min(y)))**2)

    mu.append(a)
    muerr.append(np.sqrt(aerr**2 + syserr**2))
    plt.errorbar(data['P'], data[f'T{i}'], xerr=P_ERR, yerr=yerr, 
                 fmt=f'{colors[i - 1][0]}.')

# Оформление
plt.xlabel('$\\Delta P$, A')
plt.ylabel('$\\Delta T$, \\textdegree C')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/T.pgf')

inverse_T = [10**3 / (t + 273) for t in T]
mu_T = pd.DataFrame()
mu_T['$T$, K'] = T
mu_T['$1/T$, $10^3$ K$^{-1}$'] = inverse_T
mu_T['$\mu$'] = mu
mu_T['$\Delta\mu$'] = muerr

plt.figure(figsize=(7, 4))
(alpha, beta) = plot_linear(inverse_T, mu, label='', color='red')
mu_T.to_csv('data/mu.csv')

plt.errorbar(inverse_T, mu, yerr=muerr, fmt='r.')
plt.xlabel('$T^{-1}$, $10^{-3}$ K$^{-1}$')
plt.ylabel('$\\mu$, $10^{-5}$')
plt.grid()
plt.savefig('data/mu.pgf')
